# Allergan - List
## Info
* Bei manchen Adressen wird location nicht korrekt exportiert. Manuell erfassen!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("bericht-2018.pdf", pages='all', lattice=True)

## Format Table

In [21]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty']

#Copy empty in total, if not empty
df_export.loc[df_export['empty'].notna(), 'total'] = df_export['empty']

#Add Type
index_hco = df_export[df_export['name'].str.contains(" one line per HCO", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove column empty
df_export = df_export.drop(columns=['empty'])

#Remplace N(/A
df_export['donations_grants'] = df_export['donations_grants'].str.replace('N(/A', '', regex=False)

#Remove empty
df_export = df_export.dropna(subset=['total'], how='all')

#Convert to Numbers
df_export = amounts_to_number(df_export)


#Remove Carination
df_export = remove_carination(df_export)

#Add fields
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'allergan')

saved


In [ ]:
#write_to_excel(df_export, 'tmp.xlsx', open=True)